In [39]:
from openai import OpenAI
import os
from dotenv import load_dotenv

In [ ]:
def get_api_key():
    # Retrieve the API key from the environment variables
    load_dotenv()

    api_key = os.getenv("OPENAI_API_KEY")
    # Check if the API key is present
    if not api_key:
        raise ValueError("API key not found in the .env file")
    
    return api_key

In [26]:
key=get_api_key()
client = OpenAI(api_key=key)


## Data Ingestions

In [16]:
def saveFileOpenAI(location):
    with open(location, 'rb') as file:
        response = client.files.create(file=file, purpose='assistants')
    print(response)
    return response.id

In [2]:
import os

# Base directory
base_dir = os.getcwd()

# Additional location to add
additional_location = "data/DemoDFL.pptx"

# Combine paths using os.path.join()
loc = os.path.join(base_dir, additional_location)

# Display the result
print("Base Directory:", base_dir)
print("Additional Location:", additional_location)
print("New Path:", loc)


Base Directory: c:\Users\faiza\Videos\Bayes\GPT\Assistant_OpenAI
Additional Location: file/DemoDFL.pptx
New Path: c:\Users\faiza\Videos\Bayes\GPT\Assistant_OpenAI\file/DemoDFL.pptx


In [17]:
file_id=saveFileOpenAI(location=loc)


FileObject(id='file-DmCJ5aXW1QY7yVueYG0K9COw', bytes=934888, created_at=1704824894, filename='Research_2.pdf', object='file', purpose='assistants', status='processed', status_details=None)


'file-DmCJ5aXW1QY7yVueYG0K9COw'

## Create Assistants

In [21]:
def start_assistants():
    assistant = client.beta.assistants.create(
    instructions="You are knowledge assistant,Use your knowledge base to best respond to user queries",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}],
    file_ids=[file_id]
)
    print(assistant)
    return assistant.id

In [22]:
assistan_id=start_assistants()

Assistant(id='asst_TU3IMSi6vvHUzwzACFNiRO94', created_at=1704825821, description=None, file_ids=['file-DmCJ5aXW1QY7yVueYG0K9COw'], instructions='You are knowledge assistant,Use your knowledge base to best respond to user queries', metadata={}, model='gpt-4-1106-preview', name=None, object='assistant', tools=[ToolRetrieval(type='retrieval')])


'asst_TU3IMSi6vvHUzwzACFNiRO94'

## Create Threat

In [27]:
def start_tread(propmt):
    messages=[
        {
        "role": "user",
        "content": propmt,
        }
    ]
    thread = client.beta.threads.create(messages=messages
    )
    print(thread)
    return thread.id

In [28]:
question="Who is the author in research paper and what is the title of the research paper"
tread_id=start_tread(question)

Thread(id='thread_VpsnT0aJlyIXgeURB6feaUNu', created_at=1704826270, metadata={}, object='thread')


## Run Assistance

In [29]:
def run_assistance(thread_id,assistant_id):
    run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id,
    )
    print(run)
    return run.id

In [30]:
run_id=run_assistance(tread_id,assistan_id)

Run(id='run_GoAbG7wJqvyRpyU4uBqISKSt', assistant_id='asst_TU3IMSi6vvHUzwzACFNiRO94', cancelled_at=None, completed_at=None, created_at=1704826568, expires_at=1704827168, failed_at=None, file_ids=['file-DmCJ5aXW1QY7yVueYG0K9COw'], instructions='You are knowledge assistant,Use your knowledge base to best respond to user queries', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_VpsnT0aJlyIXgeURB6feaUNu', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


### checkrunstatus

In [31]:
def checkRunStatus(thread_id,run_id):
    run=client.beta.threads.runs.retrieve(run_id=run_id,thread_id=thread_id)
    return run.status

In [33]:
checkRunStatus(tread_id,run_id)

'completed'

## Retrive Message

In [35]:
def retrieveThread(thread_id):
    thread_message=client.beta.threads.messages.list(thread_id=thread_id)
    list_message=thread_message.data
    print(thread_message)
    assistance_message=list_message[0]
    reference=assistance_message.content[0].text.annotations[0].file_citation.quote
    messsage_text=assistance_message.content[0].text.value


    return messsage_text,reference

In [36]:
mess_text,refer=retrieveThread(thread_id=tread_id)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_2agnPOoAcF4o6sHIMyMzZ3Ej', assistant_id='asst_TU3IMSi6vvHUzwzACFNiRO94', content=[MessageContentText(text=Text(annotations=[TextAnnotationFileCitation(end_index=182, file_citation=TextAnnotationFileCitationFileCitation(file_id='file-DmCJ5aXW1QY7yVueYG0K9COw', quote='Scalability and Reliability in Semi-Decentralized\nFederated Learning With Blockchain: Trust\nPenalization and Asynchronous Functionality\nline 1: 1st Given Name Surname \nline 2: dept. name of organization\n(of Affiliation) \nline 3: name of organization \n(of Affiliation) \nline 4: City Country\nline 5: email address or ORCID'), start_index=172, text='【7†source】', type='file_citation'), TextAnnotationFileCitation(end_index=401, file_citation=TextAnnotationFileCitationFileCitation(file_id='file-DmCJ5aXW1QY7yVueYG0K9COw', quote='1st Given Name Surname \nline 2: dept. name of organization\n(of Affiliation) \nline 3: name of organization \n(of Affiliation) \nline 4: Ci

In [37]:
print(mess_text)

The title of the research paper is "Scalability and Reliability in Semi-Decentralized Federated Learning With Blockchain: Trust Penalization and Asynchronous Functionality"【7†source】.

The authors of the research paper are listed as follows:
- 1st Given Name Surname
- 2nd Given Name Surname
- 3rd Given Name Surname
- 4th Given Name Surname
- 5th Given Name Surname
- 6th Given Name Surname【8†source】.

Unfortunately, the specific names of the authors are not provided in the quotes extracted from the document. The placeholders "Given Name" and "Surname" are used instead of the real names. To get the actual names of the authors, I would need to access the full document content at the section listing author names. Would you like me to do that?
